In [ ]:
import tensorflow as tf
from keras.layers import Embedding, GRU, Dense, Dropout, Flatten
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from nltk import word_tokenize, sent_tokenize
import keras.utils as ku 
import pandas as pd
import numpy as np
import string, os 
import warnings
import ckip
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

2022-12-29 09:43:29.632996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/hsuchenchien/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
lyrics = pd.read_csv("PiggyLo2_new_2.csv", usecols=['lyric_ckip'])
lyrics_full = pd.read_csv("PiggyLo2_new_2.csv", usecols=['lyric_full'])
lyrics_full = lyrics_full.values.tolist()
stop_sign = ['[', "'", ']', '，', ' ', '。', ',']
lyrics = lyrics.values.tolist()

<class 'list'>


"['電影',"

In [ ]:
lyrics_clean = []
for lines in lyrics:
    lines_temp = lines[0][2:len(lines[0]) - 2]
    lines_split = lines_temp.split("', '")
    for i in range(len(lines_split)):
        if lines_split[i] not in stop_sign:
            lyrics_clean.append(lines_split[i])  

In [ ]:
seg_len = 5

segments = []
next_chars = []
for j in range(0, len(lyrics_clean) - seg_len):
    segments.append(lyrics_clean[j:j + seg_len])
    next_chars.append(lyrics_clean[j + seg_len])

In [ ]:
#construct one-hot vectors
vocab = sorted(list(set(lyrics_clean)))
vocab_size = len(vocab)

segments_vector = np.zeros((len(segments), seg_len, vocab_size), int)
next_chars_vector = np.zeros((len(segments), vocab_size), int)
for i, segment in enumerate(segments):
    for j, char in enumerate(segment):
        segments_vector[i, j, vocab.index(char)] = 1
    next_chars_vector[i, vocab.index(next_chars[i])] = 1

In [ ]:
#GRU model
model1 = Sequential()
model1.add(GRU(16, input_shape=(seg_len, vocab_size)))
model1.add(Dense(vocab_size, activation='softmax'))
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 16)                214800    
                                                                 
 dense (Dense)               (None, 4457)              75769     
                                                                 
Total params: 290,569
Trainable params: 290,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.optimizers import Adam
optimizer =  Adam()
model1.compile(loss='categorical_crossentropy', optimizer=optimizer)

model1.fit(segments_vector, next_chars_vector, batch_size=5, epochs=16)

Epoch 1/16
9444/9444 [==============================] - 61s 6ms/step - loss: 6.7935
Epoch 2/16
9444/9444 [==============================] - 67s 7ms/step - loss: 6.1872
Epoch 3/16
9444/9444 [==============================] - 68s 7ms/step - loss: 5.6304
Epoch 4/16
9444/9444 [==============================] - 69s 7ms/step - loss: 5.1255
Epoch 5/16
9444/9444 [==============================] - 65s 7ms/step - loss: 4.7062
Epoch 6/16
9444/9444 [==============================] - 65s 7ms/step - loss: 4.3596
Epoch 7/16
9444/9444 [==============================] - 65s 7ms/step - loss: 4.0705
Epoch 8/16
9444/9444 [==============================] - 65s 7ms/step - loss: 3.8264
Epoch 9/16
9444/9444 [==============================] - 65s 7ms/step - loss: 3.6176
Epoch 10/16
9444/9444 [==============================] - 65s 7ms/step - loss: 3.4319
Epoch 11/16
9444/9444 [==============================] - 65s 7ms/step - loss: 3.2709
Epoch 12/16
9444/9444 [==============================] - 65s 7ms/step - lo

In [ ]:
def sample(pred, temperature):
    pred = pred ** (1 / temperature)
    pred = pred / np.sum(pred)
    return np.argmax(pred)

In [ ]:
import random
import sys
import jieba

"""
被遮起來的部分都是為了一次跑很多結果製作csv檔而設的，現在留下來的部分就只會隨機跑一個結果，由random.randint決定
如果想要設一個種子給使用者輸入的話可以再放這裡
"""
# keyword = []
# answer_sen = []
text_len = 10  # 這邊只設定要輸出10*2個字而已，若需要的話可以再增加
temperature = 0.3  # 溫度統一設成0.3
check = 0  # 控制答案只會生成一個
while check == 0:
    target_index = random.randint(0, len(lyrics_full) - 1)  # 種子放在這
    generated_text = lyrics_full[target_index][0][:5]

    generated_text_cut = jieba.lcut(generated_text)
    try:
        answer = generated_text
        next_anal = generated_text_cut
        for k in range(2):  #for-loop 圈數控制總共要生成幾個字 
            while len(answer) < text_len * (k + 1):
                sampled = np.zeros((1, seg_len, vocab_size), int)
                for j, word in enumerate(next_anal):
                    sampled[0, j, vocab.index(word)] = 1
                pred = model1.predict(sampled, verbose=0)[0]  # 預測下一個字
                next_index = sample(pred, temperature)
                next_char = vocab[next_index]
                answer += str(next_char)j
                next_anal.append(next_char)
                next_anal.pop(0)
            answer += '。'
        sys.stdout.write(answer)
#         keyword.append(''.join(generated_text))
#         answer_sen.append(answer)
        check += 1  # 生成成功，結束迴圈
    except:
        check = 0  # 生成失敗的話就重跑一遍

您的電話將能怎麼要掌握。你愛你看愛說是的。

In [ ]:
# import csv
# with open('answerStatEpoch16.csv', mode='w', encoding='utf-8') as outcome:
#     writer = csv.writer(outcome)
#     writer.writerow(["keyword","output with temp = 0.3"])
#     for i in range(len(keyword)):
#         writer.writerow([keyword[i], answer_sen[i]])